# Coronagraph inpainting model

## Data downloading and preprocessing

In [ ]:
!python3 data.py

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
Importing data
LASCO-C2:20031028_201035.fits:2003/10/28T20:10:35.655T...
Traceback (most recent call last):
  File "/home/jenci/Desktop/DL-based-Coronagraph-Inpainting/data.py", line 72, in <module>
    downloader.level_1()
  File "/home/jenci/Desktop/DL-based-Coronagraph-Inpainting/data.py", line 67, in level_1
    level_1(path, path)
  File "/home/jenci/.local/lib/python3.10/site-packages/corkit/lasco.py", line 58, in level_1
    b, header = c2_calibrate(img0, header, **kwargs)
  File "/home/jenci/.local/lib/python3.10/site-packages/corkit/lasco.py", line 484, in c2_calibrate
    header, expfac, bias = get_exp_factor(header) #change for python imp
  File "/home/jenci/.local/lib/python3.10/site-packages/corkit/utils.py", line 69, in get_exp_factor
    date = datetime.strptime(header['date-obs'], '%Y/%m/%d')
  File "/usr/lib/python3.10/_

In [1]:
from utils import get_default_device
device = get_default_device()
print(device)

cuda


## Creating dataset

In [2]:
batch_size = 1

In [3]:
from torch.utils.data import random_split, DataLoader
from utils import DeviceDataLoader
from data import CoronagraphDataset

dataset = CoronagraphDataset('c2')
# 0.8 - 0.2
train_len = round(0.8*len(dataset))
val_len = len(dataset) - train_len

#random split
train_ds, val_ds = random_split(dataset, [train_len, val_len])

train_dl = DeviceDataLoader(DataLoader(train_ds, batch_size, True, num_workers = 4, pin_memory=True), device)
val_dl = DeviceDataLoader(DataLoader(val_ds, batch_size*2, True, num_workers = 4, pin_memory=True), device)

/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
from model import CoronagraphReconstructor
from utils import to_device

model = to_device(CoronagraphReconstructor('c2', in_channels = 1, deep = 2, layers = 2), device)

1. c2_1_4_0.pt
2. c2_1_4_15.pt


In [5]:
import torch
sample_tensor = (torch.randn(1,1,1024,1024), torch.ones(1,1,1024,1024))
sample_tensor = to_device(sample_tensor, device)

In [6]:
torch.cuda.empty_cache()

In [7]:
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim import Adam

model.fit(
    train_dl,
    val_dl,
    epochs = 25,
    lr = 1e-5,
    batch_size = batch_size,
    weight_decay = 0,
    grad_clip = 1e-4,
    opt_func = Adam,
    lr_sched = OneCycleLR,
    saving_div = 5,
    graph = True,
    sample_input = sample_tensor
)

Training - Epoch: 5:  58%|█████▊    | 338/579 [02:45<01:57,  2.05it/s]


KeyboardInterrupt: 